In [1]:
from api_wrappers.kraken_wrapper import KrakenWrapper

wrapper = KrakenWrapper(lb_interval="1day")

all_assets = wrapper.get_assets("USD")
print(len(all_assets))
for asset in all_assets:
    print(asset + "\n")

424
1INCHUSD

AAVEUSD

ACAUSD

ACHUSD

ACTUSD

ACXUSD

ADAUSD

ADXUSD

AEROUSD

AEVOUSD

AGLDUSD

AI16ZUSD

AIRUSD

AIXBTUSD

AKTUSD

ALCHUSD

ALCXUSD

ALGOUSD

ALICEUSD

ALPHAUSD

ALTUSD

ANKRUSD

ANLOGUSD

ANONUSD

APENFTUSD

APEUSD

API3USD

APTUSD

APUUSD

ARBUSD

ARCUSD

ARKMUSD

ARPAUSD

ARUSD

ASTRUSD

ATHUSD

ATLASUSD

ATOMUSD

AUCTIONUSD

AUDIOUSD

AUDUSD

AVAAIUSD

AVAXUSD

AXSUSD

B3USD

BABYUSD

BADGERUSD

BALUSD

BANANAS31USD

BANDUSD

BATUSD

BCHUSD

BEAMUSD

BERAUSD

BICOUSD

BIGTIMEUSD

BIOUSD

BITUSD

BLURUSD

BLZUSD

BMTUSD

BNBUSD

BNCUSD

BNTUSD

BOBAUSD

BODENUSD

BONDUSD

BONKUSD

BRICKUSD

BSXUSD

BTTUSD

C98USD

CAKEUSD

CATUSD

CELOUSD

CELRUSD

CFGUSD

CHEEMSUSD

CHRUSD

CHZUSD

CLANKERUSD

CLOUDUSD

CLVUSD

COMPUSD

COOKIEUSD

CORNUSD

COTIUSD

COWUSD

CPOOLUSD

CQTUSD

CROUSD

CRVUSD

CSMUSD

CTSIUSD

CVCUSD

CVXUSD

CXTUSD

CYBERUSD

DAIUSD

DASHUSD

DBRUSD

DEGENUSD

DENTUSD

DOGSUSD

DOLOUSD

DOTUSD

DRIFTUSD

DRVUSD

DUCKUSD

DYDXUSD

DYMUSD

EDGEUSD

EG

In [ ]:
hist_dict = wrapper.pull_kraken_hist_usd_simple()

test_key = list(hist_dict.keys())[0]

[[1686096000, '0.3330', '0.3330', '0.3110', '0.3150', '0.3215', '78673.31962527', 213], [1686182400, '0.3130', '0.3280', '0.3120', '0.3230', '0.3200', '163364.35005819', 213], [1686268800, '0.3230', '0.3270', '0.3180', '0.3210', '0.3235', '61898.17245826', 142], [1686355200, '0.3230', '0.3230', '0.2360', '0.2740', '0.2765', '580777.19030879', 665], [1686441600, '0.2730', '0.2760', '0.2660', '0.2690', '0.2701', '113260.87043387', 171], [1686528000, '0.2690', '0.2740', '0.2600', '0.2700', '0.2700', '145256.06290500', 181], [1686614400, '0.2700', '0.2770', '0.2640', '0.2680', '0.2703', '39456.89022471', 79], [1686700800, '0.2670', '0.2780', '0.2580', '0.2630', '0.2671', '90428.52191796', 114], [1686787200, '0.2630', '0.2650', '0.2540', '0.2610', '0.2616', '41231.66633577', 66], [1686873600, '0.2600', '0.2790', '0.2590', '0.2760', '0.2677', '15008.99924719', 65], [1686960000, '0.2730', '0.2840', '0.2730', '0.2770', '0.2794', '16373.88101780', 69], [1687046400, '0.2760', '0.2840', '0.2750',

In [6]:
test_result = hist_dict[test_key]

for row in test_result:
    print(row[-2])

78673.31962527
163364.35005819
61898.17245826
580777.19030879
113260.87043387
145256.06290500
39456.89022471
90428.52191796
41231.66633577
15008.99924719
16373.88101780
24546.52831757
7598.87078050
32714.80382923
27718.51927404
50372.87039234
183255.36374789
32779.33093164
114071.39002489
17879.25532237
9468.54589818
100616.89883461
17691.87482177
43671.41057290
16661.65388118
10173.40103662
17849.40876381
10816.51036278
61174.78609219
35599.57249282
60223.72681818
45439.05151740
8889.31197039
18763.34793794
14869.49170348
133045.66525334
114134.97282729
151260.05813945
443732.00887945
761578.49948850
2031152.56687146
376737.03118521
251729.34914189
326082.81252712
70422.15863667
38406.89483692
27308.29541420
180611.06513280
38741.05250020
412103.48849801
50129.62461331
10060.39577797
37649.49031158
33568.75883997
45022.81844601
36717.71198219
35589.78501856
35031.87879066
30741.83946889
23951.71030224
37156.02027802
96573.41900068
51785.13369917
27653.77552554
24365.05874645
25616.819